### 전처리 순서
sol 1. 이미지에 패딩을 추가하여 정사각 이미지로 만듬 : 버그만 늘고 별 의미는 없는 것 같아서 sol 2. 로 변경
1. padding_and_resizing : 코드 정리와 파일 및 폴더 경로 업데이트 안된 상태
2. format_translation : trans_format_with_padding 함수 사용
3. split_dataset

sol 2. 
1. selective_resizing.ipynb
2. format_translation : trans_format_without_padding 함수 사용
3. split_dataset

In [70]:
# from PIL import Image
import cv2
import numpy as np
import os

# 이미지 패딩 및 리사이즈 함수
def pad_and_resize(image_path, # .jpg or .jpeg
                   output_path, 
                   target_size=(640, 640)):
    
    if not os.path.exists(image_path):
        print(f"Error: The file {image_path} does not exist.")
        return
    
    # 한글 경로는 인식못하므로 cv2.imread() 가 안된다
    img_array = np.fromfile(image_path, np.uint8)
    # BGR 
    image_BGR = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

    # image = cv2.imread(img_path)

    if image_BGR is None:
            print(f"Error: Failed to load image from {image_path}")
            return
    # BGR ->  RGB
    image = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2RGB)
    
    height, width = image.shape[0:2]
    # height, width = image.shape[0:2]

    padding = [np.abs(height - width) // 2, np.abs(height - width) // 2]

    # 부족한 길이가 절반으로 안 떨어질 경우 +1
    if np.abs(height-width) % 2 != 0:
        padding[0] += 1

    # 가로, 세로 가운데 부족한 쪽에 margin 추가
    if height < width:
        margin_list = [padding, [0, 0]]
    else:
        margin_list = [[0, 0], padding]

    # color 이미지일 경우 color 채널 margin 추가
    if len(image_BGR.shape) == 3:
        margin_list.append([0,0])

    # 이미지에 margin 추가
    padded_image = np.pad(image, margin_list, mode='constant')

    resized_image = cv2.resize(padded_image, target_size)

    ## 필요시 이미지 exif 메타데이터의 orientation 값을 보정하기 위한 rotation
    ## open cv 라이브러리를 쓰는 지금의 경우라면 필요없다.
    # image = Image.open(image_path)
    # exif_data = image._getexif()
    # orientation = exif_data.get(274) # orientation tag 는 274 번

    # # print("orientation : ", orientation) # test

    # if orientation == 6:
    #      output = cv2.rotate(resized_image, cv2.ROTATE_90_CLOCKWISE)
    # elif orientation == 8:
    #      output= cv2.rotate(resized_image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    # elif orientation == 3:
    #      output= cv2.rotate(resized_image, cv2.ROTATE_180)
    # else :
    #      output = resized_image

    # view
    # cv2.imshow('Resized Image', resized_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    # save
    # 한글 경로 문제 핸들링
    output_RGB = cv2.cvtColor(resized_image, cv2.COLOR_RGB2BGR)
    result, encoded_img = cv2.imencode('.jpg', output_RGB)  # 확장자 .jpg
    if result:
        encoded_img.tofile(output_path)  # np.tofile로 한글 경로에 저장
    else:
        print("이미지 인코딩 실패")

    #save
    # output_RGB = cv2.cvtColor(resized_image, cv2.COLOR_RGB2BGR)
    # success = cv2.imwrite(output_path, output_RGB)
    # if success:
    #     pass
    # else:
    #     print(output_path)
    #     print("이미지 저장 실패")

### test code ###
images = ["2_09_1_1_1_1_20210716_0000037039.jpg", #samsung
          "2_09_1_1_1_1_20210719_0000006002.jpg", # apple
          ]

for image in images:
    image_path = f'./원천데이터/09.점자블럭/1.불량/{image}'
    output_path = f'./{image}'

    pad_and_resize(image_path, output_path)

In [73]:
import os

root_path = "./원천데이터"

# folders = ["09.점자블럭", "12.보도블록", "13.자전거도로"]
# sub_folders= ["0.양호", "1.불량"]

folders = ["12.보도블록", "13.자전거도로"]
sub_folders= ["0.양호", "1.불량"]

# folders = ["09.점자블럭"] # test
# sub_folders= ["1.불량"] # test

for folder in folders:
    for sub_folder in sub_folders:
        image_path = f"{root_path}/{folder}/{sub_folder}"
        output_path = f"{root_path}/{folder}/{sub_folder}640"

        print(image_path)

        # image_names = [f for f in os.listdir(image_path) if f.endswith('.jpg')]
        image_names = [f for f in os.listdir(image_path) if f.endswith('.jpg') or f.endswith('.jpeg')] # .jpeg 도 극히 일부 존재
        
        total = len(image_names)

        for i, image_name in enumerate(image_names):
            if (i % 200 == 0): print(f"{i+1} / {total}")
            pad_and_resize(f"{image_path}/{image_name}", f"{output_path}/{image_name}")

./원천데이터/12.보도블록/0.양호
1 / 1416
201 / 1416
401 / 1416
601 / 1416
801 / 1416
1001 / 1416
1201 / 1416
1401 / 1416
./원천데이터/12.보도블록/1.불량
1 / 3624
201 / 3624
401 / 3624
601 / 3624
801 / 3624
1001 / 3624
1201 / 3624
1401 / 3624
1601 / 3624
1801 / 3624
2001 / 3624
2201 / 3624
2401 / 3624
2601 / 3624
2801 / 3624
3001 / 3624
3201 / 3624
3401 / 3624
3601 / 3624
./원천데이터/13.자전거도로/0.양호
1 / 1380
201 / 1380
401 / 1380
601 / 1380
801 / 1380
1001 / 1380
1201 / 1380
./원천데이터/13.자전거도로/1.불량
1 / 3540
201 / 3540
401 / 3540
601 / 3540
801 / 3540
1001 / 3540
1201 / 3540
1401 / 3540
1601 / 3540
1801 / 3540
2001 / 3540
2201 / 3540
2401 / 3540
2601 / 3540
2801 / 3540
3001 / 3540
3201 / 3540
3401 / 3540


### 누락된 파일이 없는지 갯수 확인  

In [74]:
folders = ["09.점자블럭", "12.보도블록", "13.자전거도로"]
sub_folders= ["0.양호", "1.불량"]

root_path = "./원천데이터"
# 특정 경로 설정
for folder in folders:
    for sub_folder in sub_folders:
        path = f"{root_path}/{folder}/{sub_folder}"
        output_path = f"{root_path}/{folder}/{sub_folder}640"

        print("\n",path)
        # json_files = [f for f in os.listdir(path) if f.endswith('.jpg')]
        json_files = [f for f in os.listdir(path) if f.endswith('.jpg') or f.endswith('.jpeg')]
        json_total = len(json_files)
        print("original json file counts : ", json_total)

        print("\n", output_path)
        txt_files = [f for f in os.listdir(output_path) if f.endswith('.jpg') or f.endswith('.jpeg')]
        txt_total = len(txt_files)
        print("txt file counts : ", txt_total)


 ./원천데이터/09.점자블럭/0.양호
original json file counts :  504

 ./원천데이터/09.점자블럭/0.양호640
txt file counts :  504

 ./원천데이터/09.점자블럭/1.불량
original json file counts :  1578

 ./원천데이터/09.점자블럭/1.불량640
txt file counts :  1578

 ./원천데이터/12.보도블록/0.양호
original json file counts :  1416

 ./원천데이터/12.보도블록/0.양호640
txt file counts :  1416

 ./원천데이터/12.보도블록/1.불량
original json file counts :  3624

 ./원천데이터/12.보도블록/1.불량640
txt file counts :  3624

 ./원천데이터/13.자전거도로/0.양호
original json file counts :  1380

 ./원천데이터/13.자전거도로/0.양호640
txt file counts :  1380

 ./원천데이터/13.자전거도로/1.불량
original json file counts :  3540

 ./원천데이터/13.자전거도로/1.불량640
txt file counts :  3540


### 이미지 사이즈 탐색  
각자 다를 수 있음  

In [12]:
import cv2
import os
import numpy as np

# root_path = "./원천데이터" 

# folders = ["09.점자블럭", "12.보도블록", "13.자전거도로"]
# sub_folders= ["0.양호", "1.불량"]

root_path = "D:/Downloads/street-facilities-raw/images"

folders = ["09.braille-block", "12.sidewalk-block", "13.bicycle-road"]
sub_folders= ["bad", "good"]

# folders = ["09.braille-block"]
# sub_folders= ["bad"]


for folder in folders:
    for sub_folder in sub_folders:
        
        widths = set()
        heights = set()
        edge_case_cnt = 0

        image_path = f"{root_path}/{folder}/{sub_folder}"
        # image_names = [f for f in os.listdir(image_path) if f.endswith('.jpg')] 
        image_names = [f for f in os.listdir(image_path) if f.endswith('.jpg') or f.endswith('.jpeg')] # .jpeg 도 극히 일부 존재

        for image_name in image_names:
            # img = Image.open(f"{image_path}/{image_name}")
            # width, height = img.size

            # 한글 경로는 인식못하므로 cv2.imread() 가 안된다
            img_array = np.fromfile(f"{image_path}/{image_name}", np.uint8)
            # BGR 
            image_BGR = cv2.imdecode(img_array, cv2.IMREAD_COLOR)

            # image = cv2.imread(img_path)

            if image_BGR is None:
                    print(f"Error: Failed to load image from {image_path}")
        
            # BGR ->  RGB
            image = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2RGB)
            height, width = image.shape[0:2]

            heights.add(height)
            widths.add(width)
            
            # edge case 검사
            # !((height == 2268 and width == 4032) or (height == 4032 and width == 2268))
            if ((height != 2268 or width != 4032) and (height != 4032 or width != 2268)): 
                # print(f"({width}, {height})")
                # print(image_name)
                edge_case_cnt += 1

        print("edge cases : ", edge_case_cnt)
        print("total : ", len(image_names))

        print(widths)
        print(heights)

edge cases :  459
total :  1578
{2178, 1440, 2592, 1314, 1836, 2604, 2988, 1980, 1470, 2248, 2250, 2252, 2128, 2005, 2268, 1759, 2272, 2160, 2173}
{4032, 2560, 2848, 4000, 3264, 3840, 3520, 4608, 5312, 3565, 4624, 1589, 2614, 3862, 3128, 3871}
edge cases :  322
total :  504
{2592, 1440, 2252, 2604, 2988, 2322, 2230, 2268}
{4608, 4000, 4032, 2560, 5312, 4128, 4624, 3964}
edge cases :  754
total :  3624
{1536, 4608, 1800, 1419, 2061, 2322, 1941, 1815, 1182, 1440, 2592, 1314, 1697, 2340, 2091, 1836, 2604, 2988, 2248, 2250, 1996, 2252, 720, 2128, 2267, 2268, 2272, 2023, 1903, 2160}
{2560, 3840, 1280, 4608, 3718, 3594, 3597, 4624, 3226, 4000, 2848, 4128, 2731, 3381, 4030, 3264, 4032, 4160, 5312, 3018, 2128, 3665, 2524, 3549}
edge cases :  333
total :  1416
{2176, 2187, 2193, 2322, 1440, 2592, 1836, 2604, 2988, 1845, 1080, 2248, 2250, 2252, 3024, 2256, 2128, 2140, 2268, 2272, 2160, 1919}
{4032, 2560, 3840, 3264, 4000, 1920, 4608, 4128, 5312, 3887, 4624, 3279, 3413, 3899, 3868, 3805}
edge cas

애플 디바이스 사진이랑
삼성 디바이스 사진이랑  
EXIF 태그 중 하나인 orientation 값 차이로 존재  

orientation
ref. https://feel5ny.github.io/2018/08/06/JS_13/

사진을 resize 하면서 기존 이미지 파일에 있던 이미지 메타데이터가 사라진다.  
원래 사진이 orientation : 6 일 경우,  
이미지 뷰어나 cv2 라이브러리에서는 내부적으로 orientation 값을 바탕으로 처리.  
즉, orientation : 6 인 이미지는 이미지 뷰어에서 옆으로 누운 이미지가 아니라 위로 바로 선 이미지로 보여진다는 뜻.  

전처리를 거친 이미지는 orientation 값이 None 이 되어서 다음 둘 중 하나로 방법을 정해야 될 듯 하다.
1. orientation 값을 이전 이미지의 값으로 추가  
2. 회전  -> 아래 코드  

In [42]:
from PIL import Image
from PIL.ExifTags import TAGS
import os

image_path = './원천데이터/09.점자블럭/1.불량'
# images = ['2_09_0_1_6_1_20211026_0000763010.jpg', '2_09_0_1_6_2_20210909_0000493166.jpg']

# images = [f for f in os.listdir(image_path) if f.endswith('.jpg') or f.endswith('.jpeg')]
images = ["2_09_1_1_1_1_20210717_0000000343.jpg"]

for img in images:
    image = Image.open(f"{image_path}/{img}")

    # EXIF 데이터 가져오기
    exif_data = image._getexif()

    # EXIF 태그 확인
    if exif_data:
        # for tag, value in exif_data.items():
        #     tag_name = TAGS.get(tag, tag)
        #     if (tag_name == "Model"):
        #         print(f"{tag_name}: {value}")

        orientation = exif_data.get(274)  # Orientation 태그는 274번

        if orientation == 6:
            print(f"{image_path}/{img}")
            print(exif_data.get(272)) # Model
            print("orientation : ", orientation)

        if orientation == None: # 로테이션 필요없음, 애플 디바이스
            pass
        elif orientation == 1: # 로테이션 필요없음, 애플 디바이스
            pass  # Normal
        elif orientation == 3: # 확인 안됨
            image = image.rotate(180, expand=True)
        elif orientation == 6: # 로테이션 필요, 삼성 디바이스
            image = image.rotate(270, expand=True)
        elif orientation == 8: # 로테이션 필요, 삼성 디바이스
            image = image.rotate(90, expand=True)

        # # 이미지 저장
        # image.save(f'./{img}')

./원천데이터/09.점자블럭/1.불량/2_09_1_1_1_1_20210717_0000000343.jpg
SM-A505N
orientation :  6
